# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210105_223141_cgan_b_sigmafloat_m2', '20201220_073…

In [11]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210108_124647_cgan_b_sigmafloat_3classes_m2


## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
46866,46866.0,14.0,0.139584,0.136711,0.276296,3.595908,86.530769,82.934860,-2.413665,82.567764,-2.837047,3.891027,-3.703309,-3.535563,0.139393
46867,46867.0,14.0,0.098970,0.256192,0.355162,5.251605,86.317368,81.065765,-2.621773,81.743851,-2.748200,3.724022,-2.558940,-5.240807,0.141108
46868,46868.0,14.0,0.238858,0.052541,0.291400,3.161216,85.009781,81.848564,-2.066737,80.232986,-2.734519,2.610384,-4.573561,-3.033977,0.142885
46869,46869.0,14.0,0.172898,0.354152,0.527050,5.898643,87.070824,81.172180,-2.403073,82.222786,-2.662033,4.020938,-1.864894,-5.894055,0.147044
46870,46870.0,14.0,0.349249,0.095831,0.445079,2.795006,81.681152,78.886147,-2.579562,81.708145,-2.516033,1.894407,-4.560703,-2.632988,0.140348
46871,46871.0,14.0,0.151301,0.323196,0.474497,5.465047,87.332054,81.867004,-1.522804,81.666031,-2.180931,4.184482,-2.230046,-5.457029,0.140804
46872,46872.0,14.0,0.345758,0.098015,0.443773,2.425810,83.492523,81.066711,-1.901935,82.425735,-1.556630,1.785878,-4.825202,-2.221955,0.141910
46873,46873.0,14.0,0.103455,0.358352,0.461807,6.539267,87.355507,80.816238,-1.979800,82.093155,-1.450769,5.996666,-1.455948,-6.536953,0.141883
46874,46874.0,14.0,0.330785,0.107042,0.437827,3.200607,83.023903,79.823296,-2.295781,83.073944,-1.871672,1.528443,-5.310009,-3.073905,0.146334
46875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.302139,-1.922411,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [15]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.5567151069641114


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
33160,33160.0,10.0,0.369719,0.101377,0.471096,2.753951,81.122780,78.368828,-2.902754,78.893204,-2.991827,1.822873,-4.976879,-2.592592,0.143250
34787,34787.0,11.0,0.295031,0.136009,0.431040,5.420412,86.053223,80.632812,-2.487820,82.110397,-2.975370,2.063796,-6.000873,-5.402982,0.135299
46796,46796.0,14.0,0.617240,0.110841,0.728081,1.416352,83.435822,82.019470,-2.007143,80.138840,-2.972571,0.921968,-5.677457,-0.846717,0.143782
44999,44999.0,14.0,0.176461,0.113090,0.289551,3.823545,82.808975,78.985428,-1.985259,82.501968,-2.972244,3.279350,-4.503347,-3.768080,0.157187
40749,40749.0,13.0,0.083089,0.270594,0.353684,6.606514,87.088783,80.482269,-2.508630,84.178909,-2.963862,4.269937,-1.936082,-6.603159,0.134212
44344,44344.0,14.0,0.094935,0.324221,0.419156,6.438560,87.052879,80.614319,-2.769122,81.994614,-2.963171,5.326640,-1.767392,-6.435802,0.140599
44345,44345.0,14.0,0.396621,0.127335,0.523955,2.377341,83.399590,81.022247,-2.634222,81.776375,-2.959447,1.565007,-4.752046,-2.176188,0.141121
36113,36113.0,11.0,0.159452,0.170363,0.329815,3.818488,83.259247,79.440758,-2.641587,81.236374,-2.959201,3.075145,-2.955753,-3.775786,0.153385
43392,43392.0,13.0,0.154871,0.295821,0.450692,5.459575,86.805130,81.345558,-2.092185,81.040169,-2.955568,4.368937,-1.828966,-5.452716,0.134149
38329,38329.0,12.0,0.082309,0.131577,0.213886,3.518999,83.825226,80.306229,-2.572217,80.630424,-2.955166,4.550243,-3.649745,-3.455731,0.144067


In [16]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [17]:

df_list=[]
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=np.arange(len(sigma_lst))

for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list



In [40]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
        print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        print(suffix)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']

                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
                yerr=np.sqrt((yerr2**2)/(y1**2) + (yerr1**2) * (yerr2**2/(yerr1**4)))/np.sqrt(row['num_imgs'+suffix])

#                 plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            
                plt.legend()
            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,:,:]
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
    
    if plot_type in ['hist','spec']:     
        plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
        plt.yscale('log')

f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(1, 79)
_3


<ipython-input-40-6533377466b7>:60: MatplotlibDeprecationWarning: The 'linthreshx' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  plt.xscale('symlog',linthreshx=50)


In [34]:
best_step

array([28890, 32580, 32700, 33820, 41440, 41690, 43930, 46490])

### Read validation data

In [19]:
# np.load(fname,mmap_mode='r+')[:10].shape


In [20]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
# sigma_list=[0.5,0.8,1.1]
bkgnd_dict={}
num_bkgnd=2000

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r+')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy


In [21]:
bkgnd_dict.keys()

dict_keys(['0.5', '0.65', '0.8', '1.1'])

In [22]:
df_merged.columns

Index(['epoch', 'step', 'img_type', 'fname_0', 'chi_1a_0', 'chi_1b_0',
       'chi_1c_0', 'chi_1_0', 'chi_2_0', 'chi_imgvar_0', 'chi_spec1_0',
       'chi_spec2_0', 'chi_spec3_0', 'num_imgs_0', 'num_large_0',
       'num_vlarge_0', 'hist_val_0', 'hist_err_0', 'hist_bin_centers_0',
       'spec_val_0', 'spec_sdev_0', 'label', 'fname_1', 'chi_1a_1', 'chi_1b_1',
       'chi_1c_1', 'chi_1_1', 'chi_2_1', 'chi_imgvar_1', 'chi_spec1_1',
       'chi_spec2_1', 'chi_spec3_1', 'num_imgs_1', 'num_large_1',
       'num_vlarge_1', 'hist_val_1', 'hist_err_1', 'hist_bin_centers_1',
       'spec_val_1', 'spec_sdev_1', 'fname_2', 'chi_1a_2', 'chi_1b_2',
       'chi_1c_2', 'chi_1_2', 'chi_2_2', 'chi_imgvar_2', 'chi_spec1_2',
       'chi_spec2_2', 'chi_spec3_2', 'num_imgs_2', 'num_large_2',
       'num_vlarge_2', 'hist_val_2', 'hist_err_2', 'hist_bin_centers_2',
       'spec_val_2', 'spec_sdev_2', 'fname_3', 'chi_1a_3', 'chi_1b_3',
       'chi_1c_3', 'chi_1_3', 'chi_2_3', 'chi_imgvar_3', 'chi_spec1_3',
  

In [23]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label_idx))
    print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

1
['sum_chi_1', 'sum_chi_spec1', 'sum_chi_1c', 'chi_1_1', 'chi_spec1_1', 'chi_1c_1']


,label,chi_1a_0,chi_1b_0,chi_1c_0,chi_1_0,chi_2_0,chi_imgvar_0,chi_spec1_0,chi_spec2_0,chi_spec3_0,...,chi_2_3,chi_imgvar_3,chi_spec1_3,chi_spec2_3,chi_spec3_3,sum_chi_1,sum_chi_spec1,sum_chi_1c,epoch,step
4143,13-41440,0.000877,0.003114,0.000216,0.004207,0.000142,14.391900,0.295176,31.562665,26.590483,...,0.002335,13.236080,0.460211,25.836271,30.907921,0.032507,1.414448,0.010897,13,41440
4168,13-41690,0.004020,0.002759,0.000494,0.007273,0.000309,13.022466,0.547978,47.697419,27.048056,...,0.001606,12.404100,0.446173,37.940546,29.888807,0.036853,1.847564,0.010353,13,41690
4392,14-43930,0.002548,0.002242,0.000410,0.005200,0.001333,13.494866,0.234357,24.917149,25.945240,...,0.000950,12.548534,0.849096,41.674269,30.681313,0.037879,1.836517,0.011033,14,43930
4652,14-46530,0.003364,0.003005,0.000396,0.006766,0.000710,15.174635,0.494089,26.035535,29.700031,...,0.002045,15.246170,0.628020,24.155816,33.642772,0.038998,2.093165,0.009085,14,46530
3552,11-35530,0.005050,0.004090,0.000568,0.009708,0.003884,11.942808,0.368371,37.697990,27.205058,...,0.000156,12.722348,0.557383,31.692280,27.149012,0.039254,1.658712,0.010764,11,35530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,12-38210,0.052786,0.021793,0.000407,0.074986,0.006775,16.181989,0.589549,17.278861,31.956124,...,0.009184,14.306493,0.770713,32.622355,32.863179,0.192793,1.966040,0.025120,12,38210
4413,14-44140,0.000877,0.002791,0.000719,0.004387,0.000707,13.960631,0.350874,33.178477,27.883096,...,0.016554,16.505061,0.510512,18.241523,35.180570,0.194457,1.357169,0.035059,14,44140
4357,13-43580,0.010527,0.006545,0.000838,0.017910,0.009203,14.467325,0.477689,37.134084,30.608719,...,0.009966,15.269226,0.519849,18.713182,35.090396,0.206885,1.847781,0.020339,13,43580
4599,14-46000,0.004305,0.003213,0.000217,0.007734,0.001168,13.959954,0.215722,22.085565,27.586941,...,0.002633,16.987728,0.598205,12.030994,36.069664,0.212103,1.399800,0.029088,14,46000


In [24]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [25]:
df_sliced=w.result
# df_sliced

In [26]:
best_step=[]
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_spec1',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

['sum_chi_1', 'sum_chi_spec1', 'sum_chi_1c', 'chi_1_0', 'chi_1_1', 'chi_1_2', 'chi_1_3', 'chi_spec1_0', 'chi_spec1_1', 'chi_spec1_2', 'chi_spec1_3', 'chi_1c_0', 'chi_1c_1', 'chi_1c_2', 'chi_1c_3']
['sum_chi_1', 'sum_chi_spec1', 'sum_chi_1c', 'chi_1_0', 'chi_1_1', 'chi_1_2', 'chi_1_3', 'chi_spec1_0', 'chi_spec1_1', 'chi_spec1_2', 'chi_spec1_3', 'chi_1c_0', 'chi_1c_1', 'chi_1c_2', 'chi_1c_3']
['sum_chi_1', 'sum_chi_spec1', 'sum_chi_1c', 'chi_1_0', 'chi_1_1', 'chi_1_2', 'chi_1_3', 'chi_spec1_0', 'chi_spec1_1', 'chi_spec1_2', 'chi_spec1_3', 'chi_1c_0', 'chi_1c_1', 'chi_1c_2', 'chi_1c_3']
[28890 32580 32700 33820 41440 41690 43930 46490]


In [27]:
# best_step=[6176]
# best_step=np.arange(40130,40135).astype(int)

In [28]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 79)
[(9, 28890), (10, 32580), (10, 32700), (10, 33820), (13, 41440), (13, 41690), (14, 43930), (14, 46490)]


In [29]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
2888,9-28890,0.101934,0.570002,0.014148
3257,10-32580,0.086042,2.637422,0.006747
3269,10-32700,0.511618,0.572449,0.177749
3381,10-33820,0.126800,1.680511,0.006008
4143,13-41440,0.032507,1.414448,0.010897
4168,13-41690,0.036853,1.847564,0.010353
4392,14-43930,0.037879,1.836517,0.011033
4648,14-46490,0.034872,1.750462,0.012235


In [30]:
df_best['chi_1_3']

2888    0.031403
3257    0.035452
3269    0.210147
3381    0.026605
4143    0.015014
4168    0.012281
4392    0.017079
4648    0.012903
Name: chi_1_3, dtype: float64

In [41]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=(0.5, 0.65, 0.8, 1.1), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type)>

In [ ]:
bkgnd_dict['1.1']